# Example Usage of Scraipe Library

Here's a quick example using scraipe to extract mentions of celebrities in news articles.

## Setup
Install and import things we need. We will use `NewsScraper` and `OpenAiAnalyzer` from the `scraipe[extras]` subpackage.

We will also load [your OpenAI API key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key) from a file called 'openai_key.txt'. This file should be in the same folder as this notebook.

In [ ]:
# Install Scraipe
%pip install scraipe[extras] --quiet --upgrade
# Check package version
!pip show scraipe | grep Version

Note: you may need to restart the kernel to use updated packages.
Version: 0.1.24


In [7]:

# Import modules
import pandas as pd
from scraipe import Workflow
from scraipe.extras import NewsScraper, OpenAiAnalyzer
from pydantic import BaseModel

# Load OpenAI API key
OPENAI_API_KEY = open("openai_key.txt").read().strip()

## Extract links
First, we need a list of links to target with scraipe. We will extract all links from the front page of https://apnews.com.

In [8]:
import requests
import re

url = "https://apnews.com/"
response = requests.get(url)
html = response.text

# Use a regex to extract article links
pattern = r'href=["\'](?=[^"\']*/article)([^"\']+)["\']'
news_links = re.findall(pattern, html)

# Remove duplicates
news_links = list(set(news_links))

# Display a summary of the links
news_links_df = pd.DataFrame(news_links, columns=['link'])
import time
print(f"Found {len(news_links_df)} front page AP News links on {time.strftime('%Y-%m-%d')}")
display(news_links_df.head())

Found 135 front page AP News links on 2025-04-04


,link
0,https://apnews.com/article/nasa-webb-telescope...
1,https://apnews.com/article/hoosiers-devries-co...
2,https://apnews.com/article/el-salvador-deporta...
3,https://apnews.com/article/south-korea-martial...
4,https://apnews.com/article/china-movies-imax-u...


## Configure Workflow
Now we'll configure the scraipe workflow using `NewsScraper` and `OpenAiAnalyzer`.

`NewsScraper` uses the `trafilatura` to extract article content from a news site without all the HTML clutter.

`OpenAiAnalyzer` uses OpenAI models to extract data from the article content.

In [9]:
#===Configure NewsScraper===
# NewsScraper doesn't require any additional configuration
scraper = NewsScraper()

#===Configure OpenAiAnalyzer===
# Define the instruction for the LLM. Ensure the instruction specifies a return schema.
instruction = '''
Extract a list of celebrities mentioned in the article text.
Return a JSON dictionary with the following schema:
{"celebrities":["celebrity1", "celebrity2", ...]}
'''

# (Optional) Create a pydantic schema to validate the LLM output
from typing import List
class ExpectedOutput(BaseModel):
    celebrities: List[str]
    
# Create the analyzer with the API key, instruction, and schema
analyzer = OpenAiAnalyzer(OPENAI_API_KEY,instruction,pydantic_schema=ExpectedOutput)

#===Create Workflow===
# Create a workflow with the configured scraper and analyzer
workflow = Workflow(scraper, analyzer)

## Scrape content from news links
Next we will scrape content from news links. This content will be saved within the workflow's scrape store.

In [10]:
# Scrape the news links
workflow.scrape(news_links)
# Display the scraped content
scrape_store_df = workflow.get_scrapes()
display(scrape_store_df.head())

Scraping 135/135 new or retry links...


Scraping: 100%|██████████| 135/135 [00:17<00:00,  7.69link/s]

Successfully scraped 135/135 links.


,link,content,scrape_success,scrape_error
0,https://apnews.com/article/china-eeuu-arancele...,China impondrá arancel del 34% a las importaci...,True,None
1,https://apnews.com/article/bryan-kohberger-uni...,Prosecution adds former US attorney to Univers...,True,None
2,https://apnews.com/article/tunisia-snails-alte...,"In Tunisia, snails inch toward replacing red m...",True,None
3,https://apnews.com/article/trump-national-secu...,Top Democrats protest after reported firing of...,True,None
4,https://apnews.com/article/minecraft-movie-rev...,Movie Review: Jason Momoa shines in ‘A Minecra...,True,None


## Analyze content with OpenAI
Next we will analyze the stored scrapes.

In [11]:
# Analyze the scraped content
workflow.analyze()

# Display the analyses
analysis_store_df = workflow.get_analyses()
display(analysis_store_df.head())

Analyzing 135/135 new or retry links with content...


Analyzing: 100%|██████████| 135/135 [00:53<00:00,  2.53item/s]

Successfully analyzed 135/135 links.


,link,output,analysis_success,analysis_error
0,https://apnews.com/article/china-eeuu-arancele...,{'celebrities': ['Donald Trump']},True,None
1,https://apnews.com/article/bryan-kohberger-uni...,{'celebrities': ['Joe Biden']},True,None
2,https://apnews.com/article/tunisia-snails-alte...,{'celebrities': []},True,None
3,https://apnews.com/article/trump-national-secu...,"{'celebrities': ['Donald Trump', 'Laura Loomer...",True,None
4,https://apnews.com/article/minecraft-movie-rev...,"{'celebrities': ['Jason Momoa', 'Jennifer Cool...",True,None


## Compile the results
Finally, let's export the completed analysis. 

In [14]:
export_df = workflow.export()
display(export_df)
export_df.to_csv('celebrities.csv', index=False)

,link,scrape_success,analysis_success,celebrities
0,https://apnews.com/article/china-eeuu-arancele...,True,True,[Donald Trump]
1,https://apnews.com/article/bryan-kohberger-uni...,True,True,[Joe Biden]
2,https://apnews.com/article/tunisia-snails-alte...,True,True,[]
3,https://apnews.com/article/trump-national-secu...,True,True,"[Donald Trump, Laura Loomer, Elon Musk, Pete H..."
4,https://apnews.com/article/minecraft-movie-rev...,True,True,"[Jason Momoa, Jennifer Coolidge, Jack Black, E..."
...,...,...,...,...
130,https://apnews.com/article/villanova-willard-n...,True,True,"[Kevin Willard, Scott Van Pelt, Len Elmore, Bu..."
131,https://apnews.com/article/nintendo-switch-2-m...,True,True,[]
132,https://apnews.com/article/south-africa-drugs-...,True,True,[]
133,https://apnews.com/article/hurricane-names-hel...,True,True,[]


## Analyze the results
Now you can conduct your own analysis on the structured data collected by the scraipe workflow.

In [15]:
# Load the extracted data
celebrities_df = pd.read_csv('celebrities.csv')
from ast import literal_eval
celebrities_df['celebrities'] = celebrities_df['celebrities'].apply(literal_eval)

# Explode the nested list of celebrities
celebrities_df = celebrities_df.explode('celebrities')
celebrities_df['celebrities'] = celebrities_df['celebrities'].str.strip()

# Display the top 10 most mentioned celebrities
celebrities_df = celebrities_df['celebrities'].value_counts().reset_index()
celebrities_df.columns = ['celebrity', 'mentions']
celebrities_df = celebrities_df.sort_values('mentions', ascending=False)
celebrities_df.head(10)

,celebrity,mentions
0,Donald Trump,42
1,Joe Biden,12
2,Elon Musk,7
4,Marco Rubio,5
3,Vladimir Putin,5
5,Yoon Suk Yeol,4
6,Mike Johnson,3
8,Laura Loomer,3
9,Ursula von der Leyen,3
10,Lee Jae-myung,3
